# Running Multiple Kriging Tasks

This notebook demonstrates how to run multiple kriging compute tasks concurrently
using the `run` function with a list of parameters. This is useful for:

- Scenario analysis with different parameters
- Sensitivity studies varying neighborhood settings
- Batch processing multiple attributes

All kriging results are stored as attributes in a single Block Model.

## Authentication

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

In [ ]:
# Load the widgets extension for rich HTML display
%load_ext evo.widgets

## Load Source PointSet and Variogram

Load the source pointset and variogram using `object_from_uuid`.

In [ ]:
from evo.objects.typed import object_from_uuid

# Load by UUID (replace with your actual UUIDs)
source_pointset = await object_from_uuid(manager, "9100d7dc-44e9-4e61-b427-159635dea22f")
# Alternative: load by path
# source_pointset = await object_from_path(manager, "path/to/pointset.json")

# Display the pointset (pretty-printed in Jupyter)
source_pointset

In [ ]:
# View the pointset attributes
source_pointset.attributes

In [ ]:
# Load the variogram
variogram = await object_from_uuid(manager, "72cd9b83-90f4-4cb0-9691-95728e3f9cbb")

# Alternative: load by path
# variogram = await object_from_path(manager, "path/to/variogram.json")

# Display the variogram (pretty-printed in Jupyter)
variogram

In [ ]:
print(f"Source: {source_pointset.name}")
print(f"Variogram: {variogram.name}")

## Create Target Block Model

Create a single Block Model to hold all scenario results as attributes.
The Block Model Service manages concurrent attribute creation.

In [ ]:
import uuid

from evo.blockmodels.typed import Units
from evo.objects.typed import BlockModel, Point3, RegularBlockModelData, Size3d, Size3i

run_uuid = uuid.uuid4()

# Create a Block Model to hold all scenario results
# Adjust origin, n_blocks, and block_size to match your data domain
bm_data = RegularBlockModelData(
    name=f"Kriging Scenarios - {run_uuid}",
    description="Block model with kriging results for different max_samples scenarios",
    origin=Point3(x=10000, y=100000, z=200),
    n_blocks=Size3i(nx=40, ny=40, nz=40),
    block_size=Size3d(dx=25.0, dy=25.0, dz=10.0),
    coordinate_reference_system="EPSG:32632",
    size_unit_id=Units.METRES,
)

block_model = await BlockModel.create_regular(manager, bm_data)

print(f"Created Block Model: {block_model.name}")
print(f"Block Model UUID: {block_model.block_model_uuid}")
print(f"Bounding Box: {block_model.bounding_box}")

In [ ]:
# Use existing block model instead
# lock_model = await object_from_uuid(manager, "9e19c1e7-3a52-452a-978f-73dc9440dbbe")

In [ ]:
# Display the block model (pretty-printed in Jupyter)
block_model

## Define Kriging Scenarios

Create multiple parameter sets varying the `max_samples` parameter to study its effect.
All scenarios target the same Block Model, creating different attributes.

Each scenario also uses `BlockDiscretisation` to subdivide target blocks into 3×3×2 sub-cells
for volume-averaged block kriging. Omit `block_discretisation` (or pass `None`) for point kriging.

In [ ]:
from evo.compute.tasks import BlockDiscretisation, SearchNeighborhood
from evo.compute.tasks.kriging import KrigingParameters

# Define different max_samples values to test
max_samples_values = [5, 10, 15, 20]
# Get ellipsoid from the variogram structure with largest range (default)
var_ell = variogram.get_ellipsoid()

# Create search ellipsoid by scaling the variogram ellipsoid by 2x
search_ellipsoid = var_ell.scaled(2.0)

In [ ]:
# Visualize variogram and search ellipsoids with pointset data
import plotly.graph_objects as go

# Get pointset data for center calculation and scatter plot
pts = await source_pointset.to_dataframe()
center = (pts["x"].mean(), pts["y"].mean(), pts["z"].mean())

# Generate mesh surface points for visualization
vx, vy, vz = var_ell.surface_points(center=center)
sx, sy, sz = search_ellipsoid.surface_points(center=center)

# Build visualization
var_mesh = go.Mesh3d(x=vx, y=vy, z=vz, alphahull=0, opacity=0.3, color="blue", name="Variogram Ellipsoid")
search_mesh = go.Mesh3d(x=sx, y=sy, z=sz, alphahull=0, opacity=0.2, color="gold", name="Search Ellipsoid (2x)")
scatter = go.Scatter3d(
    x=pts["x"],
    y=pts["y"],
    z=pts["z"],
    mode="markers",
    marker=dict(size=2, color=pts["Ag_ppm Values"], colorscale="Viridis", showscale=True),
    name="Sample Points",
)

fig = go.Figure(data=[var_mesh, search_mesh, scatter])
fig.update_layout(title="Kriging Inputs: Variogram & Search Ellipsoids", scene=dict(aspectmode="data"), showlegend=True)
fig.show()

In [ ]:
# Base source configuration
source = source_pointset.attributes["Ag_ppm Values"]

# Block discretisation subdivides each target block into nx * ny * nz sub-cells.
# The kriged value is averaged across these sub-cells, producing a more accurate
# volume-averaged estimate (block kriging) compared to point kriging (no discretisation).
discretisation = BlockDiscretisation(nx=3, ny=3, nz=2)

# Create parameter sets for each scenario, all targeting the same Block Model
# Note: method defaults to ordinary kriging, so we don't need to specify it
parameter_sets = []
for max_samples in max_samples_values:
    params = KrigingParameters(
        source=source,
        target=block_model.attributes[f"Samples={max_samples}"],
        variogram=variogram,
        search=SearchNeighborhood(ellipsoid=search_ellipsoid, max_samples=max_samples),
        block_discretisation=discretisation,
    )
    parameter_sets.append(params)
    print(f"Prepared scenario with max_samples={max_samples}")

print(f"\nCreated {len(parameter_sets)} parameter sets")

## Run Multiple Kriging Tasks

Execute all scenarios concurrently using `run` with a list of parameters.
Progress is aggregated across all tasks.

In [ ]:
from evo.compute.tasks import run

# Run all scenarios in parallel (progress feedback is shown by default)
print(f"Submitting {len(parameter_sets)} kriging tasks in parallel...")

results = await run(manager, parameter_sets, preview=True)

print(f"\nAll {len(results)} scenarios completed!")

## View Block Model Attributes

Display the block model attributes to see all the newly created scenario columns.

> **Note:** The block model object needs to be refreshed to see the newly added attributes.

In [ ]:
# Refresh the block model to see the new attributes added by kriging
block_model = await block_model.refresh()

# Pretty-print the block model to see its current state
block_model

In [ ]:
# View just the attributes (pretty-printed table in Jupyter)
block_model.attributes

## Query Results from Block Model

Get all scenario results from the Block Model.

In [ ]:
# Query the Block Model for all scenario columns using to_dataframe()
scenario_columns = [f"Samples={ms}" for ms in max_samples_values]

print("Querying Block Model for results...")
df = await block_model.to_dataframe(columns=scenario_columns)

print(f"Retrieved {len(df)} blocks with {len(scenario_columns)} scenario columns")
df.head(10)

## Display Results

In [ ]:
# Pretty-print the Block Model with all scenarios (includes Portal/Viewer links)
block_model

In [ ]:
# Show individual job result messages
for i, (job_result, max_samples) in enumerate(zip(results, max_samples_values)):
    print(f"Scenario {i + 1}: max_samples={max_samples} - {job_result.message}")

In [ ]:
# Display first result (pretty-printed)
results[0]

## Analyze Results

Compare the kriging results across different max_samples values.

In [ ]:
# Show statistics for each scenario
print("Statistics by max_samples:")
print(df[scenario_columns].describe())

In [ ]:
# Optional: Visualize the differences using plotly
try:
    import plotly.express as px

    # Melt the data for box plot comparison
    df_melted = df[scenario_columns].melt(var_name="Scenario", value_name="value")

    fig = px.box(
        df_melted,
        x="Scenario",
        y="value",
        title="Kriging Values by Max Samples",
    )
    fig.show()
except ImportError:
    print("Install plotly for visualization: pip install plotly")

## Create a Report on the Block Model

After running kriging, we can create a resource report on the block model.

Reports require:
1. Columns to have units defined
2. At least one category column for grouping results

### Add a Domain Column

First, let's add a category column for grouping. We'll create simple geological domains
by slicing the block model into three zones based on elevation (z-coordinate).

In [ ]:
# Get block model data
df = await block_model.to_dataframe()

# Create domain column based on z-coordinate (elevation)
# Divide into 3 domains: LMS1 (lower), LMS2 (middle), LMS3 (upper)
z_min, z_max = df["z"].min(), df["z"].max()
z_range = z_max - z_min


def assign_domain(z):
    if z < z_min + z_range / 3:
        return "LMS1"  # Lower zone
    elif z < z_min + 2 * z_range / 3:
        return "LMS2"  # Middle zone
    else:
        return "LMS3"  # Upper zone


df["domain"] = df["z"].apply(assign_domain)

# Add the domain column to the block model
domain_data = df[["x", "y", "z", "domain"]]
version = await block_model.add_attribute(domain_data, "domain")
print(f"Added domain column. New version: {version.version_id}")

# Check domain distribution
print("\nDomain distribution:")
print(df["domain"].value_counts())

In [ ]:
# Refresh to see the new attribute
block_model = await block_model.refresh()
block_model.attributes

### Set Units on Kriged Attributes

Reports require columns to have units defined. The kriged columns may not have units set,
so we need to set them before creating a report.

In [ ]:
from evo.blockmodels.typed import Units

# Set units on the kriged attribute columns
# Use the first scenario column name as an example
first_scenario_col = scenario_columns[0]

block_model = await block_model.set_attribute_units(
    {
        first_scenario_col: Units.GRAMS_PER_TONNE,  # Set appropriate unit for your data
    }
)
print(f"Set units on {first_scenario_col}")

# View updated attributes
block_model.attributes

### Create and Run the Report

Now we can create a report specification that will calculate tonnages and grades by domain.

**Key classes for reports:**
- `Aggregation` - Enum: `MASS_AVERAGE` (for grades), `SUM` (for metal content)
- `Units` - Constants for output units (e.g., `Units.GRAMS_PER_TONNE`)
- `MassUnits` - Constants for mass output (e.g., `MassUnits.TONNES`)

In [ ]:
from evo.blockmodels.typed import (
    Aggregation,
    MassUnits,
    ReportCategorySpec,
    ReportColumnSpec,
    ReportSpecificationData,
)

# Define the report
report_data = ReportSpecificationData(
    name="Kriging Results Report",
    description="Resource estimate by domain using kriged grades",
    columns=[
        ReportColumnSpec(
            column_name=first_scenario_col,
            aggregation=Aggregation.MASS_AVERAGE,  # Use MASS_AVERAGE for grades
            label="Kriged Grade",
            output_unit_id=Units.GRAMS_PER_TONNE,  # Use Units class for discoverability
        ),
    ],
    categories=[
        ReportCategorySpec(
            column_name="domain",
            label="Domain",
            values=["LMS1", "LMS2", "LMS3"],
        ),
    ],
    mass_unit_id=MassUnits.TONNES,  # Use MassUnits class
    density_value=2.7,  # Fixed density (or use density_column_name)
    density_unit_id=Units.TONNES_PER_CUBIC_METRE,
    run_now=True,  # Run immediately
)

# Create the report
report = await block_model.create_report(report_data)
print(f"Created report: {report.name}")

In [ ]:
# Pretty-print the report (shows BlockSync link)
report

### View Report Results

Get the report results (waits if report is still running).

In [ ]:
# Get the latest report result
result = await report.refresh()

# Pretty-print the result (displays table in Jupyter)
result

### Filter kriging results on a category column

Krigging can be run with a filter on a category column, which allows only for a specific area to be updated rather than the entire block model.


In [ ]:
from evo.compute.tasks import RegionFilter

# Base source configuration
source = source_pointset.attributes["Ag_ppm Values"]

# Create parameter sets for each scenario, all targeting the same Block Model
# Note: method defaults to ordinary kriging, so we don't need to specify it
parameter_sets = []
for max_samples in max_samples_values:
    params = KrigingParameters(
        source=source,
        target=block_model.attributes[f"Samples={max_samples}"],
        variogram=variogram,
        search=SearchNeighborhood(ellipsoid=search_ellipsoid, max_samples=max_samples),
        target_region_filter=RegionFilter(
            attribute=block_model.attributes["domain"],
            names=["LMS1"],  # Filter by category name
        ),
    )
    print(params.to_dict())
    parameter_sets.append(params)
    print(f"Prepared scenario with max_samples={max_samples}")

# Run all scenarios in parallel (progress feedback is shown by default)
print(f"Submitting {len(parameter_sets)} kriging tasks in parallel...")

results = await run(manager, parameter_sets, preview=True)

print(f"\nAll {len(results)} scenarios completed!")

In [ ]:
# Refresh the block model to see the new attributes added by kriging
block_model = await block_model.refresh()

# Pretty-print the block model to see its current state
block_model